<a href="https://colab.research.google.com/github/clento173/OpenSourceDataAnalysis_3-2/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import requests
import pandas as pd
import json

url = "http://openapi.seoul.go.kr:8088/4d54555a6965756e3130326d6c4c5867/json/energyUseDataSummaryInfo/1/100/"
responses = []
for year in range(2015,2016):
    for month in range(1,13):
        url_month = url + str(year) + "/" + "{:02d}".format(month)
        responses.append(requests.get(url_month).json())

data = json.loads(responses)

#key = list(responses.json().keys())[0]
#data = responses.json().get(key).get('row')

print(data)
 # key = list(response.json().keys())[0]
 # data = response.json().get(key).get('row')
 # df = pd.DataFrame(data)


#url = " http://openapi.seoul.go.kr:8088/4d54555a6965756e3130326d6c4c5867/json/energyUseDataSummaryInfo/1/100/"
#api_key = "4d54555a6965756e3130326d6c4c5867"

#response = requests.get(url)

#if response.status_code == 200:
#  print("api 호출 성공")
#  print(response.json())
#else:
#  print(f"API 호출 실패: {response.status_code}")

TypeError: the JSON object must be str, bytes or bytearray, not list